In [3]:
import pandas as pd 
import torch 
import torch.nn as nn 
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import numpy as np 
import argparse
from copy import deepcopy #Add Deepcopy for args
import visdom
vis = visdom.Visdom()

from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

print(torch.__version__)
%matplotlib inline
%pylab inline
pylab.rcParams['figure.figsize'] = (15, 9)

Setting up a new session...


1.4.0
Populating the interactive namespace from numpy and matplotlib


# 1. Data loading & Preprocessing

### 1.1 Data loading

In [4]:
cd /Users/jinsungpark/Desktop/jupyter/Data_river/original02/HP_DA

/Users/jinsungpark/Desktop/jupyter/Data_river/original02/HP_DA


In [5]:
ls #현재경로에 있는 항목 확인

DA_Down_data.xlsx  DA_Down_log.xlsx   HP_Up_data.xlsx    HP_Up_log.xlsx


In [6]:
UpStream_data = pd.read_excel('HP_Up_log.xlsx')
DownStream_data = pd.read_excel('DA_Down_log.xlsx')

In [7]:
print(UpStream_data.columns)
print(DownStream_data.columns)

Index(['Date', 'HP_DO', 'HP_BOD', 'HP_COD', 'HP_SS', 'HP_TN', 'HP_TP',
       'HP_Chl_a', 'HP_Flow', 'ChaCh_DO', 'ChaCh_BOD', 'ChaCh_COD', 'ChaCh_SS',
       'ChaCh_TN', 'ChaCh_TP', 'ChaCh_Chl_a', 'ChaCh_Flow', 'UiR_Rain',
       'UiR_Solar', 'HP_Temp', 'ChaCh_Temp'],
      dtype='object')
Index(['Date', 'DA_DO', 'DA_BOD', 'DA_COD', 'DA_SS', 'DA_TN', 'DA_TP',
       'DA_Chl_a'],
      dtype='object')


In [8]:
#날짜 인덱스화
UpData = UpStream_data.set_index('Date')
DownData = DownStream_data.set_index('Date')

In [9]:
UpData.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 265 entries, 2013-01-08 to 2019-09-23
Data columns (total 20 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   HP_DO        265 non-null    float64
 1   HP_BOD       265 non-null    float64
 2   HP_COD       265 non-null    float64
 3   HP_SS        265 non-null    float64
 4   HP_TN        265 non-null    float64
 5   HP_TP        265 non-null    float64
 6   HP_Chl_a     265 non-null    float64
 7   HP_Flow      265 non-null    float64
 8   ChaCh_DO     265 non-null    float64
 9   ChaCh_BOD    265 non-null    float64
 10  ChaCh_COD    265 non-null    float64
 11  ChaCh_SS     265 non-null    float64
 12  ChaCh_TN     265 non-null    float64
 13  ChaCh_TP     265 non-null    float64
 14  ChaCh_Chl_a  265 non-null    float64
 15  ChaCh_Flow   265 non-null    float64
 16  UiR_Rain     265 non-null    float64
 17  UiR_Solar    265 non-null    float64
 18  HP_Temp      265 non-null    fl

In [10]:
#넣고싶은 상류 항목 컬럼 선택 - TP setting
UpData = UpData.iloc[:,[0,2,3,4,5,18,12,13,15,16]]
print(UpData.columns)

Index(['HP_DO', 'HP_COD', 'HP_SS', 'HP_TN', 'HP_TP', 'HP_Temp', 'ChaCh_TN',
       'ChaCh_TP', 'ChaCh_Flow', 'UiR_Rain'],
      dtype='object')


In [11]:
# #넣고싶은 상류 항목 컬럼 선택 - TN setting
# UpData = UpData.iloc[:,[0,4,5,18,7,8,10,12,13,19,15,16,17]]
# UpData.info()

In [12]:
DownData.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 265 entries, 2013-01-08 to 2019-09-23
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   DA_DO     265 non-null    float64
 1   DA_BOD    265 non-null    float64
 2   DA_COD    265 non-null    float64
 3   DA_SS     265 non-null    float64
 4   DA_TN     265 non-null    float64
 5   DA_TP     265 non-null    float64
 6   DA_Chl_a  265 non-null    float64
dtypes: float64(7)
memory usage: 16.6 KB


In [13]:
#알고싶은 하류 항목 컬럼 넘버 넣기('Date'항목이 인덱스화 돼서 컬럼 넘버가 -1씩 됨)
Colum = 5
print(DownData.columns[Colum])

DA_TP


### 1.1 Data loading

### 1.2 Data Preprocessing(normalization)

In [14]:
from sklearn.preprocessing import MinMaxScaler

UpScaler = MinMaxScaler() #상류데이터용
DownScaler = MinMaxScaler() #하류데이터용

#나중에 결과를 DeNormalizing 하기 위해 나누어 사용 하였다.

def DeNormalize(Y, Data_name, column_num, Scaler_Type):
    
    data = Data_name
    Scaler = Scaler_Type
    
    _max = Scaler.data_max_[column_num] # 역정규화 하려는 데이터의 컬럼 번호
    _min = Scaler.data_min_[column_num] 
    
    X = Y*(_max-_min) + _min
    
    return X

In [15]:
#데이터 정규화
UpData = pd.DataFrame(UpScaler.fit_transform(UpData), columns=UpData.columns, index=UpData.index)
DownData = pd.DataFrame(DownScaler.fit_transform(DownData), columns=DownData.columns, index=DownData.index)

print(UpData.isna().sum())

HP_DO         0
HP_COD        0
HP_SS         0
HP_TN         0
HP_TP         0
HP_Temp       0
ChaCh_TN      0
ChaCh_TP      0
ChaCh_Flow    0
UiR_Rain      0
dtype: int64


#2. Data Preparation

In [16]:
class RiverDataset(Dataset):
    def __init__(self, UpData, DownData, x_frames, y_frames, start, end):
        
        self.x_frames = x_frames
        self.y_frames = y_frames
        
        self.start = start
        self.end = end

        self.UpData = UpData[start:end]
        self.DownData = DownData[start:end]

    def __len__(self):
        return len(self.UpData) - (self.x_frames + self.y_frames) + 1
    #데이터를 전처리 할때 UpData와 DownData의 길이가 동일해짐(날짜를 동일한것만 추출해야 하므로), 따라서 전체길이는 둘중 하나를 사용

    def __getitem__(self, idx):
        idx += self.x_frames

        X = self.UpData.iloc[idx-self.x_frames:idx].values
        Y = self.DownData.iloc[idx:idx+self.y_frames].values
        
        return X, Y

# 3. Model Define

In [17]:
class LSTM(nn.Module):
    
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers, batch_size, dropout, use_bn):
        super(LSTM, self).__init__()
        self.input_dim = input_dim 
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.num_layers = num_layers

        self.batch_size = batch_size
        self.dropout = dropout
        self.use_bn = use_bn 
        
        self.lstm = nn.LSTM(self.input_dim, self.hidden_dim, self.num_layers) #
        self.hidden = self.init_hidden()
        self.regressor = self.make_regressor()
        
    def init_hidden(self):
        return (torch.zeros(self.num_layers, self.batch_size, self.hidden_dim),
                torch.zeros(self.num_layers, self.batch_size, self.hidden_dim))
    
    def make_regressor(self):
        layers = []
        if self.use_bn:
            layers.append(nn.BatchNorm1d(self.hidden_dim))
        layers.append(nn.Dropout(self.dropout))
        
        layers.append(nn.Linear(self.hidden_dim, self.hidden_dim // 2))
        layers.append(nn.ReLU())
        layers.append(nn.Linear(self.hidden_dim // 2, self.output_dim))
        regressor = nn.Sequential(*layers)
        return regressor
    
    def forward(self, x):
        lstm_out, self.hidden = self.lstm(x, self.hidden)
        y_pred = self.regressor(lstm_out[-1].view(self.batch_size, -1))
        return y_pred

In [18]:
class LSTM_edit(nn.Module):
    
    def __init__(self):
        super(LSTM_edit, self).__init__()
        self.input_dim = 10
        self.hidden_dim = 32
        self.output_dim = 1
        self.num_layers = 2

        self.batch_size = 8
        self.dropout = 0.1
        self.use_bn = False
        
        self.lstm = nn.LSTM(self.input_dim, self.hidden_dim, self.num_layers) #
        self.hidden = self.init_hidden()
        self.regressor = self.make_regressor()
        
    def init_hidden(self):
        return (torch.zeros(self.num_layers, self.batch_size, self.hidden_dim),
                torch.zeros(self.num_layers, self.batch_size, self.hidden_dim))
    
    def make_regressor(self):
        layers = []
        if self.use_bn:
            layers.append(nn.BatchNorm1d(self.hidden_dim))
        layers.append(nn.Dropout(self.dropout))
        
        layers.append(nn.Linear(self.hidden_dim, self.hidden_dim // 2))
        layers.append(nn.ReLU())
        layers.append(nn.Linear(self.hidden_dim // 2, self.output_dim))
        regressor = nn.Sequential(*layers)
        return regressor
    
    def forward(self, x):
        lstm_out, self.hidden = self.lstm(x, self.hidden)
        y_pred = self.regressor(lstm_out[-1].view(self.batch_size, -1))
        return y_pred

In [19]:
# 정확도 : 예측확률을 100%로 봤을때 MAPE에 따른 오차비율을 빼줌 (100-MAPE) ##RMSE, MAPE 두개로 볼 수 있게
def MAPE(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [20]:
def RMSE(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred, multioutput='raw_values')
    return np.sqrt(mse)

In [21]:
def R2(y_true, y_pred):
    R2_score = r2_score(y_true, y_pred, multioutput='raw_values')
    return R2_score

# 4. Train, Validate, Test

In [22]:
def train(model, partition, optimizer, loss_fn, args):
    trainloader = DataLoader(partition['train'],
                             batch_size=args.batch_size,
                             shuffle=False, drop_last=True)
    model.train()
    model.zero_grad()
    optimizer.zero_grad()

    bat_siz = args.batch_size
    pred = []
    true = []
    pred_results = []
    true_results = []
    train_acc = 0.0
    train_loss = 0.0
    
    for i, (X, y) in enumerate(trainloader):

        X = X.transpose(0, 1).float().to(args.device)#파이토치는 순서가 달라서 바꿔줌
        y_true = y[:, :, Colum].float().to(args.device)
        model.zero_grad()
        optimizer.zero_grad()
        model.hidden = [hidden.to(args.device) for hidden in model.init_hidden()]

        y_pred = model(X)
        loss = loss_fn(y_pred.view(-1), y_true.view(-1))
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

        pred.append(y_pred)
        true.append(y_true)

    # ==== test 데이터 시각화를 위해 x,y데이터 저장 (배치사이즈의 텐서들을 한개씩 추가해주기) ==== #
    for i in range(len(trainloader)):
        tems1 = pred[i].view(bat_siz).cpu().detach().numpy()
        tems2 = true[i].view(bat_siz).cpu().detach().numpy()
        
        for j in range(bat_siz):
            value1 = np.exp(DeNormalize(tems1[j], DownData, Colum, DownScaler))
            value2 = np.exp(DeNormalize(tems2[j], DownData, Colum, DownScaler))
            
            pred_results.append(value1)
            true_results.append(value2)
    # ========================================================================== #   

    train_loss = train_loss / len(trainloader)
    train_acc1 = RMSE(np.array(true_results), np.array(pred_results))
    train_acc2 = R2(np.array(true_results), np.array(pred_results))
#     train_acc3 = (100 - MAPE(np.array(true_results), np.array(pred_results)))

    return model, train_loss, train_acc1[0], train_acc2[0]

In [23]:
def validate(model, partition, loss_fn, args):
    valloader = DataLoader(partition['val'],
                           batch_size=args.batch_size,
                           shuffle=False, drop_last=True)
    model.eval()

    bat_siz = args.batch_size
    pred = []
    true = []
    pred_results = []
    true_results = []
    val_acc = 0.0
    val_loss = 0.0
    
    with torch.no_grad():
        for i, (X, y) in enumerate(valloader):

            X = X.transpose(0, 1).float().to(args.device)
            y_true = y[:, :, Colum].float().to(args.device)
            model.hidden = [hidden.to(args.device) for hidden in model.init_hidden()]

            y_pred = model(X)
            loss = loss_fn(y_pred.view(-1), y_true.view(-1))

            val_loss += loss.item()

            pred.append(y_pred)
            true.append(y_true)

    # ==== test 데이터 시각화를 위해 x,y데이터 저장 (배치사이즈의 텐서들을 한개씩 추가해주기) ==== #
    for i in range(len(valloader)):
        tems1 = pred[i].view(bat_siz).cpu().detach().numpy()
        tems2 = true[i].view(bat_siz).cpu().detach().numpy()
        
        for j in range(bat_siz):
            value1 = np.exp(DeNormalize(tems1[j], DownData, Colum, DownScaler))
            value2 = np.exp(DeNormalize(tems2[j], DownData, Colum, DownScaler))
            pred_results.append(value1)
            true_results.append(value2)
    # ========================================================================== #   
    
    val_loss = val_loss / len(valloader)
    val_acc1 = RMSE(np.array(true_results), np.array(pred_results))
    val_acc2 = R2(np.array(true_results), np.array(pred_results))
#     val_acc3 = (100 - MAPE(np.array(true_results), np.array(pred_results)))

    
    return val_loss, val_acc1[0], val_acc2[0]

In [24]:
def test(model, partition, args):
    testloader = DataLoader(partition['test'],
                           batch_size=args.batch_size,
                           shuffle=False, drop_last=True)
    model.eval()

    bat_siz = args.batch_size
    pred = []
    true = []
    pred_results = []
    true_results = []
    test_acc = 0.0
    
    with torch.no_grad():
        for i, (X, y) in enumerate(testloader):
            X = X.transpose(0, 1).float().to(args.device)
            y_true = y[:, :, Colum].float().to(args.device)
            model.hidden = [hidden.to(args.device) for hidden in model.init_hidden()]

            y_pred = model(X)

            pred.append(y_pred)
            true.append(y_true)

    # ==== test 데이터 시각화를 위해 x,y데이터 저장 ==== #
    for i in range(len(testloader)):
        tems1 = pred[i].view(bat_siz).cpu().detach().numpy()
        tems2 = true[i].view(bat_siz).cpu().detach().numpy()
        
        for j in range(bat_siz):
            value1 = np.exp(DeNormalize(tems1[j], DownData, Colum, DownScaler))
            value2 = np.exp(DeNormalize(tems2[j], DownData, Colum, DownScaler))
            
            pred_results.append(value1)
            true_results.append(value2)
    # ========================================== #   

    test_acc1 =  RMSE(np.array( true_results), np.array(pred_results))
    test_acc2 =  R2(np.array( true_results), np.array(pred_results))
#     test_acc3 =  (100 - MAPE(np.array( true_results), np.array(pred_results)))
    
    return test_acc1[0], test_acc2[0], pred_results, true_results

In [25]:
def test_edit(model):
    testloader = DataLoader(testset,
                           batch_size=8,
                           shuffle=False, drop_last=True)
    print('start')
    model.eval()

    bat_siz = 8
    pred = []
    true = []
    pred_results = []
    true_results = []
    test_acc = 0.0
    
    with torch.no_grad():
        for i, (X, y) in enumerate(testloader):
            X = X.transpose(0, 1).float().to(args.device)
            y_true = y[:, :, Colum].float().to(args.device)
            model.hidden = [hidden.to(args.device) for hidden in model.init_hidden()]

            y_pred = model(X)

            pred.append(y_pred)
            true.append(y_true)

    # ==== test 데이터 시각화를 위해 x,y데이터 저장 ==== #
    for i in range(len(testloader)):
        tems1 = pred[i].view(bat_siz).cpu().detach().numpy()
        tems2 = true[i].view(bat_siz).cpu().detach().numpy()
        
        for j in range(bat_siz):
            value1 = np.exp(DeNormalize(tems1[j], DownData, Colum, DownScaler))
            value2 = np.exp(DeNormalize(tems2[j], DownData, Colum, DownScaler))
            
            pred_results.append(value1)
            true_results.append(value2)
    # ========================================== #   

    test_acc1 =  RMSE(np.array( true_results), np.array(pred_results))
    test_acc2 =  R2(np.array( true_results), np.array(pred_results))
#     test_acc3 =  (100 - MAPE(np.array( true_results), np.array(pred_results)))
    print('end')
    
    return test_acc1[0], test_acc2[0], pred_results, true_results

# 5. Experiment

In [31]:
def experiment(partition, args):

#     model = LSTM(args.input_dim, args.hid_dim, args.y_frames, args.n_layers, args.batch_size, args.dropout, args.use_bn)
    model = LSTM_edit()
    model.to(args.device)
#     loss_fn = torch.nn.MSELoss() ##loss는 mse를 사용
#     loss_fn = nn.MSELoss()
    
    if args.loss == 'MSELoss':
        loss_fn = torch.nn.MSELoss()
        loss_fn = nn.MSELoss()
    elif args.loss == 'L1Loss':
        loss_fn = torch.nn.L1Loss()
        loss_fn = nn.L1Loss()
    elif args.loss == 'PoissonNLLLoss':
        loss_fn = torch.nn.PoissonNLLLoss()
        loss_fn = nn.PoissonNLLLoss()
    elif args.loss == 'KLDivLoss':
        loss_fn = torch.nn.KLDivLoss()
        loss_fn = nn.KLDivLoss()
    elif args.loss == 'BCELoss':
        loss_fn = torch.nn.BCELoss()
        loss_fn = nn.BCELoss()
    elif args.loss == 'BCEWithLogitsLoss':
        loss_fn = torch.nn.BCEWithLogitsLoss()
        loss_fn = nn.BCEWithLogitsLoss()
    else:
        raise ValueError('In-valid LossFuction choice')
    
    
    
    if args.optim == 'SGD':
        optimizer = optim.SGD(model.parameters(), lr=args.lr, weight_decay=args.l2)
    elif args.optim == 'RMSprop':
        optimizer = optim.RMSprop(model.parameters(), lr=args.lr, weight_decay=args.l2)
    elif args.optim == 'Adam':
        optimizer = optim.Adam(model.parameters(), lr=args.lr, weight_decay=args.l2)
    else:
        raise ValueError('In-valid optimizer choice')
    
    # ===== List for epoch-wise data ====== #
    train_losses = []
    val_losses = []
    train_accs_RMSE = []
    train_accs_R2 = []
    val_accs_RMSE = []
    val_accs_R2 = []
    axis = []
    # ===================================== #
    
    ## model starting point ##    
    ts = time.time()
    model, train_loss, train_acc_RMSE, train_acc_R2 = train(model, partition, optimizer, loss_fn, args)
    val_loss, val_acc_RMSE, val_acc_R2 = validate(model, partition, loss_fn, args)
    te = time.time()

    # ====== Add Epoch Data ====== #
    train_losses.append(train_loss)
    val_losses.append(val_loss)
    train_accs_RMSE.append(train_acc_RMSE)
    val_accs_RMSE.append(val_acc_RMSE)
    train_accs_R2.append(train_acc_R2)
    val_accs_R2.append(val_acc_R2)
    # ============================ #

    # # ===== Visdom visualizing ================================================================================== #
    axis.append(0)
    
    plot1 = vis.line(Y=torch.cat((torch.Tensor(train_losses).view(-1,1), torch.Tensor(val_losses).view(-1,1)), -1),
                     X=torch.cat((torch.Tensor(axis).view(-1,1), torch.Tensor(axis).view(-1,1)), -1),
                     opts=dict(title='exp_{}_loss'.format(num), legend=['train_loss','val_loss'], showlegend=True))
    
    plot2 = vis.line(Y=torch.cat((torch.Tensor(train_accs_RMSE).view(-1,1), torch.Tensor(val_accs_RMSE).view(-1,1)), -1),
                     X=torch.cat((torch.Tensor(axis).view(-1,1), torch.Tensor(axis).view(-1,1)), -1),
                     opts=dict(title='exp_{}_acc_RMSE'.format(num), legend=['train_acc','val_acc'], showlegend=True))
    
    plot3 = vis.line(Y=torch.cat((torch.Tensor(train_accs_R2).view(-1,1), torch.Tensor(val_accs_R2).view(-1,1)), -1),
                     X=torch.cat((torch.Tensor(axis).view(-1,1), torch.Tensor(axis).view(-1,1)), -1),
                     opts=dict(title='exp_{}_acc_R2'.format(num), legend=['train_acc','val_acc'], showlegend=True))    
    # # =========================================================================================================== #
    
    print('Epoch {}, Acc_RMSE(train/val): {:2.2f}/{:2.2f}, Loss(train/val) {:2.5f}/{:2.5f}. Took {:2.2f} sec'
          .format(0, train_acc_RMSE, val_acc_RMSE, train_loss, val_loss, te-ts))
    
    for epoch in range(args.epoch-1):  # loop over the dataset multiple times
        
        ts = time.time()
        model, train_loss, train_acc_RMSE, train_acc_R2 = train(model, partition, optimizer, loss_fn, args)
        val_loss, val_acc_RMSE, val_acc_R2 = validate(model, partition, loss_fn, args)
        te = time.time()

        # ====== Add Epoch Data ====== #
        train_losses.append(train_loss)
        val_losses.append(val_loss)
        train_accs_RMSE.append(train_acc_RMSE)
        val_accs_RMSE.append(val_acc_RMSE)
        train_accs_R2.append(train_acc_R2)
        val_accs_R2.append(val_acc_R2)
        # ============================ #

        # # ===== Visdom visualizing ============================================================================== #
        axis.append(epoch+1)
        
        vis.line(Y=torch.cat((torch.Tensor(train_losses).view(-1,1), torch.Tensor(val_losses).view(-1,1)), -1),
                 X=torch.cat((torch.Tensor(axis).view(-1,1), torch.Tensor(axis).view(-1,1)), -1),
                 win=plot1, update='replace')
        
        vis.line(Y=torch.cat((torch.Tensor(train_accs_RMSE).view(-1,1), torch.Tensor(val_accs_RMSE).view(-1,1)), -1),
                 X=torch.cat((torch.Tensor(axis).view(-1,1), torch.Tensor(axis).view(-1,1)), -1),
                 win=plot2, update='replace')
        
        vis.line(Y=torch.cat((torch.Tensor(train_accs_R2).view(-1,1), torch.Tensor(val_accs_R2).view(-1,1)), -1),
                 X=torch.cat((torch.Tensor(axis).view(-1,1), torch.Tensor(axis).view(-1,1)), -1),
                 win=plot3, update='replace')
        # # ====================================================================================================== #
        
        print('Epoch {}, Acc_RMSE(train/val): {:2.2f}/{:2.2f}, Loss(train/val) {:2.5f}/{:2.5f}. Took {:2.2f} sec'
              .format(epoch+1, train_acc_RMSE, val_acc_RMSE, train_loss, val_loss, te-ts))
        
    test_acc_RMSE, test_acc_R2, Pred_data, True_data = test(model, partition, args)
    
    # ======= Add Result to Dictionary ======= #
    result = {}
    result['train_losses'] = train_losses
    result['val_losses'] = val_losses
    
    result['train_accs_RMSE'] = train_accs_RMSE
    result['train_accs_R2'] = train_accs_R2
    result['val_accs_RMSE'] = val_accs_RMSE
    result['val_accs_R2'] = val_accs_R2
#     result['train_acc'] = train_acc
#     result['val_acc'] = val_acc
    result['test_RMSE'] = test_acc_RMSE
    result['test_R2'] = test_acc_R2
    
    result['test_pred'] = Pred_data
    result['test_true'] = True_data
    
    network = model
    torch.save(network.state_dict(),'lstm05.pt')
    return result

# 6. LSTM Run

In [32]:
# ====== Random Seed Initialization ====== #
seed = 666
np.random.seed(seed)
torch.manual_seed(seed)

parser = argparse.ArgumentParser()
args = parser.parse_args("")

args.device = 'cuda' if torch.cuda.is_available() else 'cpu'

# ====== Data Loading ====== #
# args.batch_size = 8 # 2~8
args.UpData = UpData
args.DownData = DownData
args.x_frames = 4
args.y_frames = 1

# ====== Model Capacity ===== #
args.input_dim = len(UpData.columns)
args.hid_dim = 32
args.n_layers = 2

# ====== Regularization ======= #
args.l2 = 0.001
args.dropout = 0.1 
args.use_bn = False

# ====== Optimizer & Training ====== #
args.optim = 'Adam'  #SGD, RMSprop, Adam...
args.loss = 'MSELoss'#'MSELoss','L1Loss','PoissonNLLLoss','KLDivLoss','BCELoss','BCEWithLogitsLoss'
args.lr = 0.0001
args.epoch = 600


# ====== Experiment Variable ====== #
name_var1 = 'hid_dim'
name_var2 = 'batch_size'
name_var3 = 'n_layers'
name_var4 = 'lr'
name_var5 = 'epoch'
list_var1 = [32]
list_var2 = [8]
list_var3 = [2]
list_var4 = [0.0001]
list_var5 = [127]


num = 0 #초기화


trainset = RiverDataset(args.UpData, args.DownData, args.x_frames, args.y_frames, '2013-01-01', '2018-06-30')
valset = RiverDataset(args.UpData, args.DownData, args.x_frames, args.y_frames, '2018-07-01', '2019-12-31')
testset = RiverDataset(args.UpData, args.DownData, args.x_frames, args.y_frames, '2018-07-01', '2019-12-31')
partition = {'train': trainset, 'val':valset, 'test':testset}


print('size of trainset :{}'.format(len(trainset)),
      'size of valset :{}'.format(len(valset)),
      'size of testset :{}'.format(len(testset)))

for var1 in list_var1:
    for var2 in list_var2:
        for var3 in list_var3:
            for var4 in list_var4:
                for var5 in list_var5:
                    num += 1
                    setattr(args, name_var1, var1)
                    setattr(args, name_var2, var2)
                    setattr(args, name_var3, var3)
                    setattr(args, name_var4, var4)
                    setattr(args, name_var5, var5)
                    print('\n exp_{}'.format(num))
                    print('loseFunc = {}, optim={}, x_frames={}, n_layers={}, batch_size={}, hid_dim={}, epoch={}, lr={}, l2={}, dropout={}, use_bn={}'
                          .format(args.loss,args.optim,args.x_frames,args.n_layers,args.batch_size,args.hid_dim,args.epoch,args.lr,args.l2,args.dropout,args.use_bn))     

                    result = experiment(partition, deepcopy(args))

            #         print('train_acc_RMSE = {:2.2f}%, val_acc_RMSE = {:2.2f}%, test_RMSE = {:2.2f}%, test_R2 = {:2.2f}%'
            #               .format(result['train_acc_RMSE'],result['val_acc_RMSE'],result['test_RMSE'],result['test_R2']))

                    vis.text('loseFunc = {}, optim={}, x_frames={}, n_layers={}, batch_size={}, hid_dim={}, epoch={}, lr={}, l2={}, dropout={}, use_bn={}'
                             .format(args.loss,args.optim,args.x_frames,args.n_layers,args.batch_size,args.hid_dim,args.epoch,args.lr,args.l2,args.dropout,args.use_bn),
                             opts=dict(title='exp_{}_text'.format(num)))
                    # 만든 모델의 test 데이터 예측 시각화

                    predict = torch.Tensor(result['test_pred']).view(-1,1)
                    truth = torch.Tensor(result['test_true']).view(-1,1)
                    axis = torch.Tensor(range(len(result['test_pred']))).view(-1,1)

                    Y_axis = torch.cat((predict, truth), -1)
                    X_axis = torch.cat((axis, axis), -1)

                    vis.line(Y = Y_axis, X = X_axis, opts=dict(title='Result_exp_{}_RMSE[{:2.3f}]_R2[{:2.3f}]'.format(num,result['test_RMSE'],result['test_R2']),
                                                               legend=['predict','true'],
                                                               showlegend=True,
                                                               layoutopts = {'plotly': {'legend': {'x':0, 'y':0}}}))

print('All done')

size of trainset :212 size of valset :45 size of testset :45

 exp_1
loseFunc = MSELoss, optim=Adam, x_frames=4, n_layers=2, batch_size=8, hid_dim=32, epoch=127, lr=0.0001, l2=0.001, dropout=0.1, use_bn=False
Epoch 0, Acc_RMSE(train/val): 0.04/0.04, Loss(train/val) 0.10150/0.08615. Took 0.34 sec
Epoch 1, Acc_RMSE(train/val): 0.04/0.04, Loss(train/val) 0.09078/0.07722. Took 0.36 sec
Epoch 2, Acc_RMSE(train/val): 0.04/0.04, Loss(train/val) 0.08117/0.06874. Took 0.35 sec
Epoch 3, Acc_RMSE(train/val): 0.04/0.03, Loss(train/val) 0.07156/0.06148. Took 0.35 sec
Epoch 4, Acc_RMSE(train/val): 0.04/0.03, Loss(train/val) 0.06321/0.05527. Took 0.35 sec
Epoch 5, Acc_RMSE(train/val): 0.03/0.03, Loss(train/val) 0.05576/0.05012. Took 0.35 sec
Epoch 6, Acc_RMSE(train/val): 0.03/0.03, Loss(train/val) 0.05037/0.04629. Took 0.36 sec
Epoch 7, Acc_RMSE(train/val): 0.03/0.03, Loss(train/val) 0.04477/0.04389. Took 0.36 sec
Epoch 8, Acc_RMSE(train/val): 0.03/0.03, Loss(train/val) 0.04223/0.04282. Took 0.37 sec

Epoch 90, Acc_RMSE(train/val): 0.02/0.02, Loss(train/val) 0.02514/0.02285. Took 0.36 sec
Epoch 91, Acc_RMSE(train/val): 0.02/0.02, Loss(train/val) 0.02482/0.02277. Took 0.38 sec
Epoch 92, Acc_RMSE(train/val): 0.02/0.02, Loss(train/val) 0.02491/0.02279. Took 0.36 sec
Epoch 93, Acc_RMSE(train/val): 0.02/0.02, Loss(train/val) 0.02480/0.02281. Took 0.35 sec
Epoch 94, Acc_RMSE(train/val): 0.02/0.02, Loss(train/val) 0.02449/0.02272. Took 0.36 sec
Epoch 95, Acc_RMSE(train/val): 0.02/0.02, Loss(train/val) 0.02543/0.02270. Took 0.35 sec
Epoch 96, Acc_RMSE(train/val): 0.02/0.02, Loss(train/val) 0.02403/0.02265. Took 0.35 sec
Epoch 97, Acc_RMSE(train/val): 0.02/0.02, Loss(train/val) 0.02501/0.02263. Took 0.35 sec
Epoch 98, Acc_RMSE(train/val): 0.02/0.02, Loss(train/val) 0.02423/0.02258. Took 0.35 sec
Epoch 99, Acc_RMSE(train/val): 0.02/0.02, Loss(train/val) 0.02396/0.02256. Took 0.36 sec
Epoch 100, Acc_RMSE(train/val): 0.02/0.02, Loss(train/val) 0.02447/0.02251. Took 0.36 sec
Epoch 101, Acc_RMSE(

In [ ]:
print('Current CPU random seed:',torch.initial_seed())
# print('Current CUDA random seed:',torch.cuda.initial_seed())

In [ ]:
# vis.text('loseFunc = {}, optim={}, x_frames={}, n_layers={}, batch_size={}, hid_dim={}, epoch={}, lr={}, l2={}, dropout={}, use_bn={}'
#                              .format(args.loss,args.optim,args.x_frames,args.n_layers,args.batch_size,args.hid_dim,args.epoch,args.lr,args.l2,args.dropout,args.use_bn),
#                              opts=dict(title='exp_{}_text'.format(num)))

In [ ]:
# seed = 666
# np.random.seed(seed)
# torch.manual_seed(seed)

# network = LSTM(14, 32, 1, 3, 4, 0.1, False)

In [ ]:
len(UpData.columns)

In [33]:
seed = 666
np.random.seed(seed)
torch.manual_seed(seed)
modell = LSTM_edit()
modell.load_state_dict(torch.load('lstm05.pt'))

<All keys matched successfully>

In [34]:
seed = 666
np.random.seed(seed)
torch.manual_seed(seed)
RMSE_1, R2_1, Pred_1, True_1 = test_edit(modell)

start
end


In [35]:
test_data = pd.DataFrame()
test_data['test_pred'] = Pred_1
test_data['test_true'] = True_1

In [36]:
test_data.to_csv('modell_result_01.csv')

In [29]:
pwd

'/Users/jinsungpark/Desktop/jupyter/Data_river/original02/HP_DA'